In [1]:
# Source: Alexandru Tifrea and Fanny Yang, 2022.

# Python Notebook Commands
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from IPython.core.display import HTML
from IPython import display

display.display(HTML("<style>.container { width:100% !important; }</style>"))

# General math and plotting modules.
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import DEFAULT_PLOTLY_COLORS

from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

# Widget and formatting modules
import ipywidgets
from ipywidgets import interact, interactive, interact_manual, fixed, widgets
from matplotlib import rcParams

rcParams['figure.figsize'] = (10, 6)
rcParams['font.size'] = 16

# Machine Learning library.
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn import datasets
from sklearn.metrics import mean_squared_error

from utils import generate_data, generate_additional_data, repeat_experiment, compute_population_risk, get_error_vs_overparametrization
from utils import compute_test_error as compute_empirical_risk

import warnings

rcParams['figure.figsize'] = (15, 6)
rcParams['font.size'] = 20
# Change these values if the images don't fit for your screen.
figure_width = 1200
figure_height = 500

# 1) Regularized Polynomial Regression

### Regression with polynomial features

In the following we show how the estimator depends on hyperparameters like the regularization coefficient (for LASSO and ridge penalties) or the the degree of the polynomial used for the features.


Let's consider 1-dimensional data $\{(x_i, y_i)\}_{i=0}^n \subset \mathbb{R} \times \mathbb{R}$. To obtain a better feature representation for the data, we map the samples to the space of monomials of degree at most $d$, i.e. $\varphi: \mathbb{R} \rightarrow span(\{1, X, X^2, ..., X^d\})$. The maximum degree controls the complexity of the regression model: the higher the degree, the more complex the features we obtain. As you will see later in the course, regression with polynomial features is equivalent to using a polynomial kernel function.

We perform regularized regression and consider two different regularization penalties:

- LASSO penalty, i.e. minimizing $L_{\text{lasso}}(w; \lambda) := \sum_{i=0}^n (y_i - w^T\varphi(x_i))^2 + \lambda ||w||_1 $.

- ridge penalty, i.e. minimizing $L_{\text{ridge}}(w; \lambda) := \sum_{i=0}^n (y_i - w^T\varphi(x_i))^2 + \lambda ||w||_2^2 $.

Below we show the mean squared error (MSE) computed on the training points, as well as on the test points. Moreover, we print the $\ell_1$ and $\ell_2$ norms of the trained estimator.

In [2]:
def true_regression_fun(ground_truth):
  if ground_truth == "sine":
    return lambda X: np.cos(3 * np.pi * X)
  elif "poly" in ground_truth:
    coefficients = [float(coef) for coef in ground_truth.split(":")[1].split(",")]
    return lambda X: np.poly1d(coefficients[::-1])(X)
  else:
    raise RuntimeError(f"Unknown ground truth function {ground_truth}")

@ignore_warnings(category=ConvergenceWarning)
def poly_kernel_regression(ground_truth, n_samples, degree, reg_type, reg_coef, noise):
    np.random.seed(101)

    X = np.sort(np.random.uniform(0 - 1e-4, 1 + 1e-4, n_samples))
    y = true_regression_fun(ground_truth)(X) + np.random.randn(n_samples) * noise

    if reg_type == "ridge" and reg_coef > 0:
      model = Ridge(alpha=reg_coef, fit_intercept=False, solver="svd")
      model_key = "ridge"
    elif reg_type == "lasso" and reg_coef > 0:
      model = Lasso(alpha=reg_coef, fit_intercept=False, tol=1e-2, max_iter=10000)
      model_key = "lasso"
    else:
      model = LinearRegression(fit_intercept=False)
      model_key = "linearregression"
    
    clf = make_pipeline(PolynomialFeatures(degree), model)
    clf.fit(X[:, np.newaxis], y)

    X_test = np.random.uniform(0 - 1e-4, 1 + 1e-4, 100)
    train_mse = mean_squared_error(
      y_true=y,
      y_pred=clf.predict(X[:, np.newaxis])
    )
    test_mse = mean_squared_error(
      y_true=true_regression_fun(ground_truth)(X_test) + np.random.randn(100) * noise,
      y_pred=clf.predict(X_test[:, np.newaxis])
    )
    
    all_X = np.sort(np.concatenate((X_test, X)))
    fig = make_subplots(rows=1, cols=2) #, row_width=[0.15, 0.35])
    fig.add_trace(go.Scatter(x=all_X,
                             y=clf.predict(all_X[:, np.newaxis]),
                             line_width=3,
                             name="Model"),
                  row=1,
                  col=1)
    fig.add_trace(go.Scatter(x=all_X,
                             y=true_regression_fun(ground_truth)(all_X),
                             line_dash="dash",
                             line_width=3,
                             name="True function"),
                  row=1,
                  col=1)
    fig.add_trace(go.Scatter(x=X,
                             y=y,
                             mode="markers",
                             marker_size=7,
                             marker_symbol="x",
                             marker_color="black",
                             name="Samples"),
                  row=1,
                  col=1)
    fig.add_trace(go.Scatter(x=np.arange(clf[model_key].coef_.shape[0]),
                             y=np.fabs(clf[model_key].coef_),
                             line_width=3,
                             showlegend=False),
                  row=1,
                  col=2)

    fig.update_layout(
        title=f"Training MSE = {train_mse:.6}" + "<br>Test MSE" + f" = {test_mse:.6}" + "<br>" + f"l2 norm = {np.linalg.norm(clf[model_key].coef_):.2}; l1 norm = {np.linalg.norm(clf[model_key].coef_, ord=1):.2}",
        height=figure_height,
        width=figure_width,
        margin=go.layout.Margin(
            l=0,  #left margin
            r=0,  #right margin
            b=0,  #bottom margin
            t=100,  #top margin
        ),
        xaxis1_range=[0, 1],
        xaxis1_title="x",
        yaxis1_range=[-2, 2],
        yaxis1_title="y",
        xaxis2_title="Degree",
        yaxis2_title="Abs. value of coefficient",
    )
    fig.update_layout(legend=dict(
        yanchor="top",
        y=0.97,
        xanchor="left",
        x=0.37
    ))
    fig.show()


_ = interact(
    poly_kernel_regression,
    ground_truth=ipywidgets.Dropdown(options=["sine", "poly:0,-1,0,0,1"],
                                     value="sine",
                                     description='Ground truth function:',
                                     disabled=False,
                                     style={'description_width': 'initial'},
                                     continuous_update=False),
    n_samples=ipywidgets.IntSlider(value=20,
                                   min=5,
                                   max=100,
                                   step=5,
                                   description='Number of samples:',
                                   style={'description_width': 'initial'},
                                   continuous_update=False),
    degree=ipywidgets.IntSlider(value=5,
                                min=1,
                                max=15,
                                step=1,
                                description='Polynomial Degree:',
                                style={'description_width': 'initial'},
                                continuous_update=False),
    reg_type=ipywidgets.Dropdown(options=["lasso", "ridge"],
                                 value="ridge",
                                 description='Regularization type:',
                                 disabled=False,
                                 style={'description_width': 'initial'},
                                 continuous_update=False),
    reg_coef=ipywidgets.FloatSlider(value=0.,
                                    min=0,
                                    max=0.001,
                                    step=0.0001,
                                    readout_format='.4f',
                                    description='Regularization coefficient:',
                                    style={'description_width': 'initial'},
                                    continuous_update=False),
    noise=ipywidgets.FloatSlider(value=0.5,
                                 min=0,
                                 max=1,
                                 step=0.1,
                                 readout_format='.2f',
                                 description='Noise level:',
                                 style={'description_width': 'initial'},
                                 continuous_update=False),
)

interactive(children=(Dropdown(description='Ground truth function:', options=('sine', 'poly:0,-1,0,0,1'), styl…

### Impact of noise on the estimator norm

In these figures we see how increasing the noise level leads to linear estimators whose weights have higher norm. This phenomenon holds true for both the $\ell_1$ and the $\ell_2$ norm. This observation motivates regularization with a norm-based penalty (like LASSO or ridge).

In [3]:
@ignore_warnings(category=ConvergenceWarning)
def norm_increase_with_noise(n_samples, degree, reg_type):
    np.random.seed(101)
    
    ground_truth = "poly:0,-1,0,0,1"
    noise_values = np.arange(0, 0.1, 0.01)
    reg_coef_values = [0., 1e-2, 1e-1, 1]
    
    X = np.sort(np.random.rand(n_samples))
    gauss_noise = np.random.randn(n_samples)
    
    fig = make_subplots(rows=1, cols=2)
    for i, reg_coef in enumerate(reg_coef_values):
      l1_norms, l2_norms = [], []
      if reg_type == "ridge" and reg_coef > 0:
        model = Ridge(alpha=reg_coef, fit_intercept=False, solver="svd")
        model_key = "ridge"
      elif reg_type == "lasso" and reg_coef > 0:
        model = Lasso(alpha=reg_coef, fit_intercept=False, tol=1e-2, max_iter=10000)
        model_key = "lasso"
      else:
        model = LinearRegression(fit_intercept=False)
        model_key = "linearregression"
      
      for noise in noise_values:
        y = true_regression_fun(ground_truth)(X) + gauss_noise * noise
        clf = make_pipeline(PolynomialFeatures(degree), model)
        clf.fit(X[:, np.newaxis], y)
        l1_norms.append(np.linalg.norm(clf[model_key].coef_, ord=1))
        l2_norms.append(np.linalg.norm(clf[model_key].coef_, ord=2))
    
      fig.add_trace(go.Scatter(x=noise_values,
                               y=l1_norms,
                               line_width=3,
                               marker_color=DEFAULT_PLOTLY_COLORS[i],
                               name=f"${reg_type.title()}\ \lambda={reg_coef}$",
                               legendgroup=f"${reg_type.title()}\ \lambda={reg_coef}$"),
                    row=1,
                    col=1)
      fig.add_trace(go.Scatter(x=noise_values,
                               y=l2_norms,
                               line_width=3,
                               marker_color=DEFAULT_PLOTLY_COLORS[i],
                               name=f"${reg_type.title()}\ \lambda={reg_coef}$",
                               legendgroup=f"${reg_type.title()}\ \lambda={reg_coef}$",
                               showlegend=False),
                    row=1,
                    col=2)

    fig.update_layout(
        height=figure_height,
        width=figure_width,
        margin=go.layout.Margin(
            l=0,  #left margin
            r=0,  #right margin
            b=0,  #bottom margin
            t=10,  #top margin
        ),
        xaxis1_title="Noise level",
        yaxis1_title="$\ell_1\ norm$",
        xaxis2_title="Noise level",
        yaxis2_title="$\ell_2\ norm$",
    )
    fig.show()
    
_ = interact(
    norm_increase_with_noise,
    n_samples=ipywidgets.IntSlider(value=20,
                                   min=10,
                                   max=100,
                                   step=10,
                                   description='Number of samples:',
                                   style={'description_width': 'initial'},
                                   continuous_update=False),
    degree=ipywidgets.IntSlider(value=5,
                                min=1,
                                max=10,
                                step=1,
                                description='Polynomial Degree:',
                                style={'description_width': 'initial'},
                                continuous_update=False),
    reg_type=ipywidgets.Dropdown(options=["lasso", "ridge"],
                                 value="ridge",
                                 description='Regularization type:',
                                 disabled=False,
                                 style={'description_width': 'initial'},
                                 continuous_update=False),
)

interactive(children=(IntSlider(value=20, continuous_update=False, description='Number of samples:', min=10, s…

# 2) Bias-variance trade-off for ridge regression

We can estimate the bias and the variance of an estimator by sampling different training sets, and using a hold-out validation set to compute its empirical error.

The stacked area plot below illustrates the decomposition of the risk into three terms: the squared bias, the variance, and irreducible noise.

In [4]:
validation_size = 1000
num_trials = 5
snr = 1

all_noise_sigmas = [0, 0.1, 0.5, 1]


def plot_bias_variance_for_ridge(n, d, noise_sigma):
    ridge_coefficients = np.arange(0, 20, 1)
    risks, squared_biases, variances = [], [], []
    n, d = int(n), int(d)

    # Sample the validation set and one traing set for each of the trials.
    X_validation, y_validation, beta_star, Sigma = generate_data(
        n=validation_size, d=d, snr=snr, noise_sigma=noise_sigma)
    all_X, all_y = generate_additional_data(num_samples=n * num_trials,
                                            d=d,
                                            Sigma=Sigma,
                                            beta_star=beta_star,
                                            noise_sigma=noise_sigma)
    for ridge_coef in ridge_coefficients:
        validation_predictions, validation_bayes_predictions = [], []

        # Train num_trials estimators and use the validation set to estimate the bias and variance.
        for i in range(num_trials):
            start, end = i * n, (i + 1) * n
            # Compute closed form solution of the ridge regression optimization problem.
            beta_hat = 1 / (1 + ridge_coef) * np.linalg.inv(
                all_X[start:end].T
                @ all_X[start:end]) @ all_X[start:end].T @ all_y[start:end]
            validation_predictions.append(
                (X_validation @ beta_hat).reshape(-1, 1))
            validation_bayes_predictions.append(
                (X_validation @ beta_star).reshape(-1, 1))

        validation_predictions, validation_bayes_predictions = np.array(
            validation_predictions), np.array(validation_bayes_predictions)
        risks.append(np.power(validation_predictions - y_validation, 2).mean())
        variances.append(
            np.power(
                validation_predictions - validation_predictions.mean(axis=0),
                2).mean())
        squared_biases.append(
            max(0, risks[-1] - variances[-1] - noise_sigma**2))

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(x=ridge_coefficients,
                   y=np.ones_like(ridge_coefficients) * noise_sigma**2,
                   name="Irreducible noise",
                   marker_color="gray"))
    fig.add_trace(
        go.Scatter(x=ridge_coefficients, y=variances, name="Variance"))
    fig.add_trace(
        go.Scatter(x=ridge_coefficients,
                   y=squared_biases,
                   name="Bias<sup>2</sup>"))
    fig.add_trace(go.Scatter(x=ridge_coefficients, y=risks, name="Risk"))

    if noise_sigma == 0.5:
        yaxis_range = [0, 1.5]
    elif noise_sigma == 1:
        yaxis_range = [0, 2.2]
    else:
        yaxis_range = [0, 1.05]

    fig.update_layout(height=figure_height,
                      width=figure_width,
                      yaxis_range=yaxis_range,
                      yaxis_title="Risk / Bias / Variance",
                      xaxis_title="Ridge coefficient",
                      hovermode='x')
    fig.show()


_ = interact(
    plot_bias_variance_for_ridge,
    n=ipywidgets.FloatSlider(value=200,
                             min=100,
                             max=500,
                             step=10,
                             readout_format='d',
                             description='Number of samples:',
                             style={'description_width': 'initial'},
                             continuous_update=False),
    d=ipywidgets.FloatSlider(value=100,
                             min=10,
                             max=100,
                             step=10,
                             readout_format='d',
                             description='Data dimension:',
                             style={'description_width': 'initial'},
                             continuous_update=False),
    noise_sigma=ipywidgets.Dropdown(options=all_noise_sigmas,
                                    value=0.5,
                                    description='Noise level:',
                                    disabled=False,
                                    style={'description_width': 'initial'},
                                    continuous_update=True),
)

interactive(children=(FloatSlider(value=200.0, continuous_update=False, description='Number of samples:', max=…

# Effect of overparametrization for ridge regression

In [8]:
def plot_risk_vs_overparametrization(n=None,
                                     d=None,
                                     cov_type="isotropic",
                                     num_runs=1):
    assert (n is not None) or (d is not None)

    all_snr = [1]
    all_gammas = np.concatenate((
        np.arange(0.1, 2, 0.1),
        # Uncomment this line for higher overparameterization, but at the cost of
        # longer computation time.
        # np.arange(3, 10)
    ))
    params = {
        "all_gammas": all_gammas,
        "all_snr": all_snr,
        "fix_n_vary_d": (n is not None),
        "cov_type": cov_type,
        "use_ridge": True,
    }
    if n is not None:
        params["n"] = n
    else:
        params["d"] = d
    ridge_aggregated_risks = repeat_experiment(
        num_runs, get_error_vs_overparametrization, params)
    params["use_ridge"] = False
    ridgeless_aggregated_risks = repeat_experiment(
        num_runs, get_error_vs_overparametrization, params)

    fig = go.Figure()
    for snr in all_snr:
        fig.add_trace(
            go.Scatter(x=all_gammas,
                       y=ridge_aggregated_risks[snr],
                       name="Ridge regression"))
        fig.add_trace(
            go.Scatter(x=all_gammas,
                       y=ridgeless_aggregated_risks[snr],
                       name="Ridgeless regression"))
    fig.update_layout(
        height=figure_height,
        width=figure_width,
        yaxis_type="log",
        yaxis_range=[-1, 2.5],
        yaxis_title="Population risk",
        xaxis_type="log",
        xaxis_title="$\Large\gamma=d/n$",
        title=f"Fixed n={int(n)}"
        if params["fix_n_vary_d"] else f"Fixed d={int(d)}",
    )
    fig.show()

### Select the number of samples

In [9]:
_ = interact(
    lambda n, cov_type, num_runs: plot_risk_vs_overparametrization(
        n=n, cov_type=cov_type, num_runs=num_runs),
    n=ipywidgets.FloatSlider(value=100,
                             min=100,
                             max=200,
                             step=10,
                             readout_format='d',
                             description='Number of samples:',
                             style={'description_width': 'initial'},
                             continuous_update=False),
    cov_type=ipywidgets.Dropdown(options=["isotropic", "misspecified"],
                                 value="isotropic",
                                 description='Covariance model:',
                                 disabled=False,
                                 style={'description_width': 'initial'},
                                 continuous_update=False),
    num_runs=ipywidgets.Dropdown(options=[1, 3, 5, 10],
                                 value=1,
                                 description='Number of experiments:',
                                 disabled=False,
                                 style={'description_width': 'initial'},
                                 continuous_update=False),
)

interactive(children=(FloatSlider(value=100.0, continuous_update=False, description='Number of samples:', max=…

### Select the dimension of the data

In [10]:
_ = interact(
    lambda d, cov_type, num_runs: plot_risk_vs_overparametrization(
        d=d, cov_type=cov_type, num_runs=num_runs),
    d=ipywidgets.FloatSlider(value=20,
                             min=20,
                             max=1000,
                             step=10,
                             readout_format='d',
                             description='Data dimension:',
                             style={'description_width': 'initial'},
                             continuous_update=False),
    cov_type=ipywidgets.Dropdown(options=["isotropic", "misspecified"],
                                 value="isotropic",
                                 description='Covariance model:',
                                 disabled=False,
                                 style={'description_width': 'initial'},
                                 continuous_update=False),
    num_runs=ipywidgets.Dropdown(options=[1, 3, 5, 10],
                                 value=1,
                                 description='Number of experiments:',
                                 disabled=False,
                                 style={'description_width': 'initial'},
                                 continuous_update=False),
)

interactive(children=(FloatSlider(value=20.0, continuous_update=False, description='Data dimension:', max=1000…